In [1]:
import time
import hdbscan
import numpy as np
import pandas as pd
from tqdm import tqdm 
# import seaborn as sns
from src.fABBA_test import fABBA
import matplotlib.pyplot as plt
from src.cABBA_test import fABBA as cABBA
from sklearn import cluster, mixture
from src.cagg_memview import aggregate as aggregate_fc
from sklearn.metrics import mean_squared_error
from tslearn.metrics import dtw as dtw
from threadpoolctl import threadpool_limits
import warnings
plt.style.use('bmh')
warnings.filterwarnings('ignore')
np.random.seed(0)

In [2]:
N = 5000
M = 100
TOL = 0.15
ALPHA = 0.5

compression_rate =  np.zeros(M)
clustering_mse = {
    'Aggregation': np.zeros(M),
    'K-means++': np.zeros(M),
    'Spectral Clustering': np.zeros(M),
    'Gaussian Mixture': np.zeros(M),
    #'BIRCH': np.zeros(M),
    'DBSCAN': np.zeros(M),
    'HDBSCAN': np.zeros(M)
}

clustering_dtw = {
    'Aggregation': np.zeros(M),
    'K-means++': np.zeros(M),
    'Spectral Clustering': np.zeros(M),
    'Gaussian Mixture': np.zeros(M),
    #'BIRCH': np.zeros(M),
    'DBSCAN': np.zeros(M),
    'HDBSCAN': np.zeros(M)
}

clustering_time = {
    'Aggregation': np.zeros(M),
    'K-means++': np.zeros(M),
    'Spectral Clustering': np.zeros(M),
    'Gaussian Mixture': np.zeros(M),
    #'BIRCH': np.zeros(M),
    'DBSCAN': np.zeros(M),
    'HDBSCAN': np.zeros(M)
}

clustering_symbols = {
    'Aggregation': np.zeros(M, dtype=int),
    'K-means++': np.zeros(M, dtype=int),
    'Spectral Clustering': np.zeros(M, dtype=int),
    'Gaussian Mixture': np.zeros(M, dtype=int),
    #'BIRCH': np.zeros(M, dtype=int),
    'DBSCAN': np.zeros(M, dtype=int),
    'HDBSCAN': np.zeros(M, dtype=int)
}

with threadpool_limits(limits=1, user_api='blas'):
    for i in tqdm(range(M)):
        ts = np.random.rand(N)
        
        st = time.time()
        fabba = fABBA(tol=TOL, alpha=ALPHA, sorting='2-norm', scl=1, verbose=1, max_len=np.inf) 
        strings = fabba.fit_transform(ts)
        inverse_ts = fabba.inverse_transform(strings, ts[0])
        et = time.time()
        
        clustering_mse['Aggregation'][i] = mean_squared_error(ts, inverse_ts)
        clustering_dtw['Aggregation'][i] = dtw(ts, inverse_ts)
        clustering_time['Aggregation'][i] = et - st
        clustering_symbols['Aggregation'][i] = len(fabba.splist)
        
        clusters_num = fabba.centers.shape[0]

        kmeans = cluster.KMeans(n_clusters=clusters_num, random_state=1, init='k-means++')
        spectral = cluster.SpectralClustering(n_clusters=clusters_num, eigen_solver='arpack', affinity="nearest_neighbors")
        gmm = mixture.GaussianMixture(n_components=clusters_num, covariance_type='full', init_params='kmeans')
        # birch = cluster.Birch(n_clusters=2*clusters_num)
        dbscan = cluster.DBSCAN(eps=TOL, min_samples=2)
        h_dbscan = hdbscan.HDBSCAN(algorithm='best', min_samples=2, min_cluster_size=12, alpha=ALPHA)
        
        clustering_algorithms = (
            ('K-means++', kmeans),
            ('Spectral Clustering', spectral),
            ('Gaussian Mixture', gmm),
            # ('BIRCH', birch),
            ('DBSCAN', dbscan),
            ('HDBSCAN', h_dbscan)
        )
        
        compression_rate[i] = fabba.compression_rate
        
        for name, clustering in clustering_algorithms:
            try:
                st = time.time()
                cabba = cABBA(clustering=clustering.fit_predict, tol=TOL, scl=1, verbose=0, max_len=np.inf) 
                strings = cabba.fit_transform(ts)
                inverse_ts = cabba.inverse_transform(strings, ts[0])
                et = time.time()
                
                clustering_mse[name][i] = mean_squared_error(ts, inverse_ts)
                clustering_dtw[name][i] = dtw(ts, inverse_ts)
                clustering_time[name][i] = et - st
                clustering_symbols[name][i] = cabba.centers.shape[0]
                # print(name, cabba.centers.shape[0])
            except Exception as e:
                print(name, "error!", e)

  0%|          | 0/100 [00:00<?, ?it/s]

Compression: Reduced series of length 5000 to 1554 segments. Digitization: Reduced 1554 pieces to 45 symbols.


  1%|          | 1/100 [00:03<06:06,  3.70s/it]

Compression: Reduced series of length 5000 to 1393 segments. Digitization: Reduced 1393 pieces to 51 symbols.


  2%|▏         | 2/100 [00:06<05:34,  3.41s/it]

...

 98%|█████████▊| 98/100 [05:41<00:06,  3.23s/it]

Compression: Reduced series of length 5000 to 1565 segments. Digitization: Reduced 1565 pieces to 43 symbols.


 99%|█████████▉| 99/100 [05:44<00:03,  3.18s/it]

Compression: Reduced series of length 5000 to 1603 segments. Digitization: Reduced 1603 pieces to 49 symbols.


100%|██████████| 100/100 [05:48<00:00,  3.48s/it]


In [3]:
print("Average compression rate:", np.mean(compression_rate))

Average compression rate: 0.3117


In [4]:
pd.DataFrame(clustering_mse).to_csv("results/COM_MSE.csv",index=False)
pd.DataFrame(clustering_dtw).to_csv("results/COM_DTW.csv",index=False)
pd.DataFrame(clustering_time).to_csv("results/COM_RUNTIME.csv",index=False)
pd.DataFrame(clustering_symbols).to_csv("results/COM_SYMBOLS.csv",index=False)